In [1]:
import numpy as np

In [2]:
sample_size = 50
n_samples = 4*[sample_size]
size = (100, 100)
elastic_modulus = (1.3, 75)
poissons_ratio = (0.42, .22)
macro_strain = 0.001
n_phases = 2
grain_size = [(40, 2), (10, 2), (2, 40), (2, 10)]
power = [1, 2, 5, 10]

In [4]:
from pymks.datasets import make_elastic_stress_random
dataset1, stresses1 = make_elastic_stress_random(n_samples=n_samples,
                                                 size=size,
                                                 grain_size=grain_size,
                                                 elastic_modulus=elastic_modulus,
                                                 poissons_ratio=poissons_ratio,
                                                 macro_strain=macro_strain,
                                                 seed=0, power=power[0])

TypeError: make_microstructure() got an unexpected keyword argument 'power'